In [13]:
# Load required packages.
import os
import torch
import numpy as np
import pandas as pd

from plyfile import PlyData, PlyElement
from helper_functions import load_object, save_object


In [14]:
#path = 'C:/Users/thoma/Desktop/ZHAW MLS/David prod/ADLProtProject/surface'
path = 'C:/Users/david/pyproj/pyg/adl/surfaces'

In [15]:
#List file on so 
from os import walk
def list_files(mpath):
  f = []
  for (dirpath, dirnames, filenames) in walk(mpath):
      f.extend(filenames)
      break
  return(f)

In [16]:
# I think this would do the trick
files_ply = os.listdir(path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/david/pyproj/pyg/adl/surfaces'

In [17]:
def read_ply_file(folder_path,file_name,requirement):

    plydata = PlyData.read(folder_path + '/' + file_name)
    
    feature = pd.DataFrame(plydata.elements[0].data)
    vertex_indices = pd.DataFrame(plydata.elements[1].data)

    if requirement == 'feature':
        return feature
    if requirement == 'vertex':
        return vertex_indices
    if requirement == 'plyfile':
        return plydata
    if requirement == 'all' :
        return [feature,vertex_indices]

In [10]:
def read_ply_folder(folder_path):

    files_ply = list_files(path)
    features_prot = [read_ply_file(folder_path,ply_name,'feature')  for ply_name in files_ply]
    vertex_indices = [read_ply_file(folder_path,ply_name,'vertex')  for ply_name in files_ply]
    
    return features_prot,vertex_indices,files_ply


def complex_separation(files_ply,df):
    
    complexes_names = []
    complexes = []
    uniq_names= []
    for f in files_ply :
        sh = f.find('_')
        val = f[0:sh]
        if val not in uniq_names:
            uniq_names.append(val)
    for name in uniq_names:
        prot = []
        prot_names = []
        max_len = 0

        for i in range(len(files_ply)):
            sh = files_ply[i].find('_')
            val = files_ply[i][0:sh]
            if name == val:
                us = len(files_ply[i])
                len_c = len(files_ply[i][sh:us])
                if len_c > max_len:
                    prot.append(df[i])
                    prot_names.append(files_ply[i])
                    max_len = len_c
                else:
                    prot.insert(0,df[i])
                    prot_names.insert(0,files_ply[i])

        complexes.append(prot)
        complexes_names.append(prot_names)
        
    return  complexes,complexes_names
        
            

feature,vetex,ply_files = read_ply_folder(path)
complexes_feature,complexes_names = complex_separation(ply_files,feature)
print(complexes_names)
print(feature)

PlyHeaderParseError: line 1: expected one of {ply}

In [ ]:
from c_ProteinGraph import ProteinGraph
from torch_geometric.utils import dense_to_sparse

source_dir = path
dest_dir = 'C:/Users/david/pyproj/pyg/adl/surfaces_graphs'

for complex in complexes_names: 

    if len(complex) != 3: # Check if there are three files of the complex
        continue
    else:
        for file in complex[0:2]: # only do it for the subunits, not for the complex
            name = file[0:-4]

            if f'{name}.pkl' not in os.listdir(dest_dir): #Only do it for proteins that are not yet in source dir

                features, edges = read_ply_file(path, file,'all')
                    
                pos = features.loc[:,["x","y","z"]].to_numpy()

                features = features.loc[:,["charge","hbond","hphob", "iface", 'nx', 'ny', 'nz']]

                    
                ## Translate edges into a adjacency matrix and then edge_index
                triangles = np.asarray([list(triangle[0]) for _,triangle in edges.iterrows()])
                n_nodes = pos.shape[0]
                adj = np.identity(n_nodes)
                for n1, n2, n3 in triangles:
                        
                    # edge1
                    adj[n1][n2] = 1
                    adj[n2][n1] = 1 # set location in adj = 1

                    #edge2
                    adj[n1][n3] = 1
                    adj[n3][n1] = 1

                    #edge3
                    adj[n2][n3] = 1
                    adj[n3][n2] = 1

                adj = torch.from_numpy(adj)
                edge_index, _ = dense_to_sparse(adj)


                protein = ProteinGraph(features=features, pos=pos, edge_index=edge_index, name=name)
                save_object(protein, dest_dir + f'/{name}.pkl')

In [25]:
from c_ProteinGraph import ProteinGraph

path = 'C:/Users/thoma/OneDrive - ZHAW/ProteinSurfaces/surfaces_graphs'

files_graph_name = list_files(path)
files_graph = []

for f in files_graph_name:
    files_graph.append(load_object(path + '/' + f))

print(files_graph[45])


 


            Number of Nodes: 4444
            Features: (4444, 7)
            Edge_Index: (2, 31096)
            Coordinates of Nodes: (4444, 3)
            Protein Name: 1DK4_B


In [47]:

graph_prot_1 = files_graph[0]
graph_prot_2 = files_graph[45]

In [48]:
print(graph_prot_1)



            Number of Nodes: 5436
            Features: (5436, 7)
            Edge_Index: (2, 38052)
            Coordinates of Nodes: (5436, 3)
            Protein Name: 1A99_C


In [74]:
from typing import List, Optional, Tuple, Union

import torch
from torch import Tensor

from torch_geometric.typing import PairTensor
from torch_geometric.utils.mask import index_to_mask
from torch_geometric.utils.num_nodes import maybe_num_nodes



def k_hop_subgraph_perso(
    node_idx: Union[int, List[int], Tensor],
    num_hops: int,
    edge_index: Tensor,
    relabel_nodes: bool = False,
    num_nodes: Optional[int] = None,
    flow: str = 'source_to_target',
    max_nodes = int
) -> Tuple[Tensor, Tensor, Tensor, Tensor]:

    num_nodes = maybe_num_nodes(edge_index, num_nodes)

    assert flow in ['source_to_target', 'target_to_source']
    if flow == 'target_to_source':
        row, col = edge_index
    else:
        col, row = edge_index

    node_mask = row.new_empty(num_nodes, dtype=torch.bool)
    edge_mask = row.new_empty(row.size(0), dtype=torch.bool)

    if isinstance(node_idx, (int, list, tuple)):
        node_idx = torch.tensor([node_idx], device=row.device).flatten()
    else:
        node_idx = node_idx.to(row.device)

    subsets = [node_idx]

    for _ in range(num_hops):
        if len(subsets) > max_nodes:
            break
        node_mask.fill_(False)
        node_mask[subsets[-1]] = True
        torch.index_select(node_mask, 0, row, out=edge_mask)
        print(col[edge_mask[0]])
        subsets.append(col[edge_mask])
        

    subset, inv = torch.cat(subsets).unique(return_inverse=True)
    inv = inv[:node_idx.numel()]

    node_mask.fill_(False)
    node_mask[subset] = True
    edge_mask = node_mask[row] & node_mask[col]

    edge_index = edge_index[:, edge_mask]

    if relabel_nodes:
        node_idx = row.new_full((num_nodes, ), -1)
        node_idx[subset] = torch.arange(subset.size(0), device=row.device)
        edge_index = node_idx[edge_index]

    return subset, edge_index, inv, edge_mask

In [73]:
from torch_geometric.utils.subgraph import k_hop_subgraph

subset,edge_index,mapping,edege_mask = k_hop_subgraph_perso(7,3,graph_prot_2.edge_index,max_nodes=12)
print(len(subset))
#print(subset)

from f_visualize_pointcloud import visualize_pointcloud

colors_protein = np.zeros_like(graph_prot_2.pos)
colors_protein[:] = [1,0,0]
colors_protein[subset] = [0,0,1]


#visualize_pointcloud(graph_prot_2.pos, colors = colors_protein)




29


In [42]:
print(edge_index.shape)

torch.Size([2, 524])


## Test what is in a pickle file now

In [ ]:
dir = path + '/graphs'
test = load_object(dir + '/1A99_C.pkl')
print(test)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/thoma/OneDrive - ZHAW/ProteinSurfaces/graphs/1A99_C.pkl'

In [ ]:
test.num_features()

NameError: name 'test' is not defined

In [ ]:
test.num_nodes()

In [ ]:
test.features

In [ ]:
test.pos

In [ ]:
test.edge_index

In [ ]:
test.name

In [ ]:
test.patches = ((edge_index, label, center_index), (edge_index, label ), ())